In [2]:
# インポート
from sources.api.bus import *
from sources.api.train import *

/Users/x0y14/dev/reachable-area/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# データの読み込み
stops = load_stop_data("../dataset/busstpos/kanagawa/P11-22_14.geojson")
stations = load_station_data("../dataset/stations/N02-20_Station.geojson")

In [4]:
# 神奈川中央交通の、KAITを通るバス停のみを取り出す。
kait_stops = []
kait_routes = ['厚07', '厚67', '厚89']  # 神奈川工科大学に辿り着けるバス路線
for stop in stops:
    for kait_route in kait_routes:
        if (kait_route in stop.routes) and (stop not in kait_stops):
            kait_stops.append(stop)

# 小田原線の駅のみ取り出す。
odakyu_stations = []
for station in stations:
    if "小田原線" == station.line:
        odakyu_stations.append(station)

print(f"神奈川中央交通のバス停: {len(kait_stops)}, 小田原線の駅: {len(odakyu_stations)}")

神奈川中央交通のバス停: 29, 小田原線の駅: 47


In [5]:
# >> 不動産公正取引協議会連合会が定める「不動産の表示に関する公正競争規約」では、徒歩で１分間に歩く距離を「80m」として所要時間が算出されています。(出典: https://www.ur-net.go.jp/chintai/college/202310/000877.html, 2024/08/06 閲覧)

# 各バス停から800メートル以内の駅を取り出す。
stations800m = []
for station in odakyu_stations:
    station_mean_coord = station.geometry.calc_mean()
    for stop in kait_stops:
        distance_between_station_and_stop = calc_distance_m(station_mean_coord, stop.geometry.calc_mean())
        if (distance_between_station_and_stop <= 800) and station not in stations800m:
            stations800m.append(station)
print(f"いずれかのバス停から800m以内に存在する小田急の駅の数: {len(stations800m)}")
for st in stations800m:
    print(st.name)

いずれかのバス停から800m以内に存在する小田急の駅の数: 1
本厚木


In [9]:
# kait経路のバス停から10分県内のバス停を取得する(緯度経度による簡易的な距離計算)
stops_within_800m_from_kait_route_stops = {}
for stop in stops:
    for kait_stop in kait_stops:
        distance = calc_distance_m(stop.geometry.calc_mean(), kait_stop.geometry.calc_mean())
        if (distance <= 800) and (stop not in kait_stops):  # 既に路線内に組み込まれているバス停はスキップ
            if str(stop) in stops_within_800m_from_kait_route_stops:
                stops_within_800m_from_kait_route_stops[str(stop)].append(
                    (stop, kait_stop, distance)
                )
            else:
                stops_within_800m_from_kait_route_stops[str(stop)] = [
                    (stop, kait_stop, distance)
                ]

print(f"KAIT路線から10分圏内のバス停の数: {len(stops_within_800m_from_kait_route_stops)}")
for key in stops_within_800m_from_kait_route_stops.keys():
    for route in stops_within_800m_from_kait_route_stops[key]:
        stop = route[0]
        k_stop = route[1]
        dis = route[2]
        print(f"{str(stop)} ~ {str(k_stop)} : {dis}m")


KAIT路線から10分圏内のバス停の数: 46
愛川町-坂本 ~ 神奈川中央交通（株）-下川入 : 722.2279947903189m
愛川町-坂本 ~ 神奈川中央交通（株）-才戸橋 : 685.3033389103202m
神奈川中央交通（株）-坂本 ~ 神奈川中央交通（株）-下川入 : 722.2279947903189m
神奈川中央交通（株）-坂本 ~ 神奈川中央交通（株）-才戸橋 : 685.3033389103202m
神奈川中央交通（株）-荻野中学校入口 ~ 神奈川中央交通（株）-鳶尾団地 : 609.2506231631515m
神奈川中央交通（株）-上三田 ~ 神奈川中央交通（株）-下川入 : 779.8822573674843m
神奈川中央交通（株）-上三田 ~ 神奈川中央交通（株）-才戸橋 : 705.6201373998452m
神奈川中央交通（株）-上三田 ~ 神奈川中央交通（株）-あつぎ郷土博物館 : 676.0824048622727m
神奈川中央交通（株）-上三田 ~ 神奈川中央交通（株）-棚沢 : 668.9819252249737m
神奈川中央交通（株）-上三田 ~ 神奈川中央交通（株）-下谷 : 527.0964773963263m
神奈川中央交通（株）-上三田 ~ 神奈川中央交通（株）-西四ツ谷 : 614.1375268514664m
神奈川中央交通（株）-上三田 ~ 神奈川中央交通（株）-リコー前 : 573.5090023750835m
神奈川中央交通（株）-上三田 ~ 神奈川中央交通（株）-神奈川工科大学 : 685.6291700873779m
神奈川中央交通（株）-上三田 ~ 神奈川中央交通（株）-神奈川工科大学前 : 775.8099063275953m
神奈川中央交通（株）-荻野神社入口 ~ 神奈川中央交通（株）-鳶尾団地 : 414.7681937980461m
神奈川中央交通（株）-荻野神社入口 ~ 神奈川中央交通（株）-鳶尾小学校入口 : 757.3984571400666m
神奈川中央交通（株）-鳶尾四丁目 ~ 神奈川中央交通（株）-鳶尾一丁目 : 773.913004404214m
神奈川中央交通（株）-鳶尾四丁目 ~ 神奈川中央交通（株）-鳶尾団地 : 237.88432526377662m
神奈川